In [25]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(
    temperature=0.1, 
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler()
        ]
)

chef_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a world-class international chef. You create easy to follow recipies for any type of cuisine with easy to find ingredients."),
    ("human", "I want to cook {cuisine} food."),
])

chef_chain = chef_prompt | chat


In [26]:
veg_chef_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a vegeterian chef specialized on making traditional recipies vegeterian. You find alternative ingredients and explain their preparation. You don't redically modify the recipe. If there is no alternative for a food just say you don't know how to replace it."),
    ("human", "{recipe}")
])

veg_chain = veg_chef_prompt | chat 

final_chain = {"recipe": chef_chain} | veg_chain

final_chain.invoke({
    "cuisine" : "indian"
})

Great choice! Indian cuisine is known for its rich flavors and aromatic spices. Here's a recipe for a classic Indian dish called Butter Chicken:

Ingredients:
- 500g boneless chicken, cut into bite-sized pieces
- 2 tablespoons butter
- 1 onion, finely chopped
- 2 cloves of garlic, minced
- 1-inch piece of ginger, grated
- 2 teaspoons garam masala
- 1 teaspoon turmeric powder
- 1 teaspoon chili powder (adjust according to your spice preference)
- 1 cup tomato puree
- 1/2 cup heavy cream
- Salt, to taste
- Fresh cilantro leaves, for garnish

Instructions:
1. Heat the butter in a large pan over medium heat. Add the chopped onion and sauté until it turns golden brown.
2. Add the minced garlic and grated ginger to the pan. Cook for another minute until fragrant.
3. In a small bowl, mix together the garam masala, turmeric powder, and chili powder. Add this spice mixture to the pan and cook for a minute to release the flavors.
4. Add the chicken pieces to the pan and cook until they are brown

AIMessageChunk(content="To make a vegetarian version of Butter Chicken, you can replace the boneless chicken with a plant-based alternative such as tofu or paneer (Indian cottage cheese). Here's how you can modify the recipe:\n\nIngredients:\n- 500g tofu or paneer, cut into bite-sized pieces\n- 2 tablespoons butter (or vegan butter for a vegan version)\n- 1 onion, finely chopped\n- 2 cloves of garlic, minced\n- 1-inch piece of ginger, grated\n- 2 teaspoons garam masala\n- 1 teaspoon turmeric powder\n- 1 teaspoon chili powder (adjust according to your spice preference)\n- 1 cup tomato puree\n- 1/2 cup coconut cream (or vegan heavy cream alternative)\n- Salt, to taste\n- Fresh cilantro leaves, for garnish\n\nInstructions:\n1. Heat the butter in a large pan over medium heat. Add the chopped onion and sauté until it turns golden brown.\n2. Add the minced garlic and grated ginger to the pan. Cook for another minute until fragrant.\n3. In a small bowl, mix together the garam masala, turmeric